<a href="https://colab.research.google.com/github/danielpatrickhug/Sentence_Kernels/blob/main/chatgpt_lecture2QAipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!sudo apt update && sudo apt install ffmpeg


Get:1 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease [3,622 B]
Get:2 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu2004/x86_64  InRelease
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease [1,581 B]
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu2004/x86_64  Release
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  Packages [907 kB]
Hit:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease
Hit:9 http://archive.ubuntu.com/ubuntu focal InRelease
Get:10 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Get:12 http://security.ubuntu.com/ubuntu focal-security/universe amd64 Packages [1,008 kB]
Hit:13 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease
H

In [ ]:
%%capture
%%bash
pip install openai
pip install python-ffmpeg
python -m pip install pytube
pip install git+https://github.com/openai/whisper.git 
pip install setuptools-rust

In [ ]:
import openai
from getpass import getpass
import whisper
from pytube import YouTube
import ffmpeg

In [ ]:
openai_secret = getpass('Enter the secret value: ')
# Set up OpenAI API credentials
openai.api_key = openai_secret

In [ ]:
loaded_model = whisper.load_model("medium.en")
current_size = 'base'

100%|██████████████████████████████████████| 1.42G/1.42G [00:12<00:00, 126MiB/s]


In [ ]:
def inference(link):
  YouTube(link).streams.first().download()
  print("Downlaoding Video")
  yt = YouTube(link)
  return yt

def whisper_inference(yt_obje):
    options = whisper.DecodingOptions(without_timestamps=False)
    print("running transcript inference")
    results = loaded_model.transcribe(yt_obje)
    return results


def change_model(size):
  if size == current_size:
    return
  loaded_model = whisper.load_model(size)
  current_size = size

def populate_metadata(link):
  yt = YouTube(link)
  return yt.thumbnail_url, yt.title

In [ ]:
link =  "https://www.youtube.com/watch?v=rLOz9naqZMI"
YouTube(link).streams.first().download()
print("Downlaoding Video")
yt = YouTube(link)


ConnectionResetError: ignored

In [ ]:
resp = whisper_inference("/content/Leetcode Challenge with DeepMind & Mila Scientists!.3gpp")
resp

running transcript inference


RuntimeError: ignored

In [ ]:
for k in resp['segments']:
    print(k['text'])

 Welcome back.
 So we're talking about the Fourier series and how we can approximate functions f of x,
 periodic functions with an infinite sum of sines and cosines of increasingly high frequencies.
 Okay, so this is the basis of the fast Fourier transform and lots of modern techniques in compression
 and numerical solutions of differential equations.
 And in the last few lectures I've shown you how to do this with cosines and sines.
 Now I'm going to show you how to do this with complex valued functions.
 So f of x doesn't have to be real. This could be a complex valued function as well.
 And in that case what we're going to do is we're going to write this as a sum, an infinite sum,
 from k equals minus infinity to infinity, so this is a little bit different than before,
 of some complex coefficient ck e to the i kx.
 And you'll notice that I went back to two pi periodic functions defined from negative pi to pi
 because my notation will be a little bit easier here.
 And I want you to 

In [ ]:
for k, v in resp.items():
    print(v)

 Welcome back. So we're talking about the Fourier series and how we can approximate functions f of x, periodic functions with an infinite sum of sines and cosines of increasingly high frequencies. Okay, so this is the basis of the fast Fourier transform and lots of modern techniques in compression and numerical solutions of differential equations. And in the last few lectures I've shown you how to do this with cosines and sines. Now I'm going to show you how to do this with complex valued functions. So f of x doesn't have to be real. This could be a complex valued function as well. And in that case what we're going to do is we're going to write this as a sum, an infinite sum, from k equals minus infinity to infinity, so this is a little bit different than before, of some complex coefficient ck e to the i kx. And you'll notice that I went back to two pi periodic functions defined from negative pi to pi because my notation will be a little bit easier here. And I want you to recall kind o

In [100]:
def chat_gpt_inference(messages: list):
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=messages
    )
    return response

def create_prompt_message_template(text, role="user"):
    if role not in ["user", "assistant"]:
        raise ValueError("Not a valid role. Please use 'user' or 'assistant'.")
    return {"role": role, "content": text}

def compose_inference(text_block, messages):
    user_template = create_prompt_message_template(text_block, role="user")
    messages.append(user_template)
    chat_resp = chat_gpt_inference(messages)
    reply_text = chat_resp['choices'][0]['message']['content']
    assistant_template = create_prompt_message_template(reply_text, role="assistant")
    messages.append(assistant_template)
    return messages, assistant_template
    


def process_transcript(segments):
    buffer = []
    messages = [{"role": "system", "content": "You are a helpful assistant and you create educational content for college students and PHDs such as question and answer pairs. Your number one priority is to be factually correct"}]
    running_size = 0
    with open('conversation.txt', 'w') as f:
        for i, sent in enumerate(segments):
            if i % 10 == 0 and len(buffer) > 5:
                block = "\n".join(buffer)
                text_block = f"Summarize the transcript up to this point with a summary up to 5 question answer pairs and representative python code. If theres nothing interesting, just reply 'skip'. ```{block}```"
                running_size += len(text_block)
                if running_size < 4000:
                    print(running_size)
                    messages, assistant_template = compose_inference(text_block, messages)
                    f.write(f"{assistant_template['content']}\n")
                    buffer = []
                else:
                    oldest_user_reply = []
                    partition_sum = 0
                    for j, msg in enumerate(messages):
                        if msg["role"] == "user":
                            text_to_replace = messages[j]["content"]
                            partition_sum += len(text_to_replace)
                            if partition_sum > 2000:
                                break
                            else:
                                oldest_user_reply.append((j, text_to_replace))

                    if len(oldest_user_reply) >= 1:
                        for oldest in oldest_user_reply:
                            running_size -= len(oldest[1])
                            messages[oldest[0]]["content"] = ""
                    messages, assistant_template = compose_inference(text_block, messages)
                    f.write(f"{assistant_template['content']}\n")
                    buffer = []
            buffer.append(sent['text'])
    return messages


processed_messages = process_transcript(resp["segments"])

957
1934
3068
3143
3985
2862
3700


In [ ]:
for msg in processed_messages:
    print(msg)

{'role': 'system', 'content': 'You are a helpful assistant and you create educational content for college students and PHDs such as question and answer pairs. Your number one priority is to be factually correct'}
{'role': 'assistant', 'content': 'Skip.'}
{'role': 'assistant', 'content': 'The speaker is discussing the Fourier series and how it can approximate periodic functions with an infinite sum of sines and cosines of increasingly high frequencies. They previously showed how to do this with cosines and sines and now they will show how to do this with complex-valued functions. \n\nQuestion: What is the basis of the fast Fourier transform and modern techniques in compression and numerical solutions? \n\nAnswer: The basis of the fast Fourier transform and modern techniques in compression and numerical solutions is the Fourier series. \n\nQuestion: How did the speaker define the Fourier series with complex coefficient ck e to the i kx in comparison to before? \n\nAnswer: The speaker def